In [ ]:
import pickle

import joblib
import pandas as pd
import numpy as np
import gc

# with open('alexnet.fea', 'rb') as f:
#     alex_features = pickle.load(f)
        

alex_features = np.array(joblib.load('alexnet.fea'))
alex_labels = np.array(joblib.load('alexnet.labels'))

train_metadata_df = pd.read_csv('dicom_metadata_train_merged.csv') # Took it from kaggle notebooks
import torch

#alex_np_fea = [ elem[0] for elem in alex_features ]
 

mult_lab = train_metadata_df.iloc[:, 64:]

In [ ]:
from torch.utils.data import Dataset
#from torchvision.io import read_image toooo old fot that stuff



class featuresDataset(Dataset):
    def __init__(self,  features_list, labels = None, merged_df = None, transform = None, target_transform = None):
        if not merged_df:
            self.img_labels = labels
            self.use_labels = True
        else:
            self.img_labels = merged_df
        self.features_list = features_list
        self.transform = transform
        self.target_transform = target_transform
        print("Dataset loaded. Length = ",len(self.features_list))


    def __len__(self):
        return len(self.features_list)

    def __getitem__(self, idx):
        feature = self.features_list[idx]
        if self.use_labels:
            label = self.img_labels[idx]
        else:
            label = self.img_labels.iloc[idx, 64:]
        if self.transform:
            feature = self.transform(feature)
        if self.target_transform:
            label = self.target_transform(label)
        return feature, label
    
    def get_all_data(self):
        return self.features_list , self.img_labels

In [ ]:
from torchvision.transforms import ToTensor, ToPILImage


training_data = featuresDataset(
#    merged_df=train_metadata_df,
    labels = alex_labels,
    features_list=alex_features,
#    train=True,
#    download=True,
    #transform=ToTensor()
)

In [ ]:

type(training_data.get_all_data()[0][0].shape),training_data[55]

In [ ]:
train_size = int(0.1 * len(training_data))
test_size = len(training_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(training_data, [train_size, test_size])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(alex_features, alex_labels, test_size=0.90, random_state=42)

In [ ]:
# This example cant work with multi-lable.
# Need to find other solution
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2

import heapq

selected_features_indexes = set()
#from sklearn.neighbors import KNeighborsClassifier
def select_features(X_train, y_train, X_test,i,selected_features_indexes):
    # configure to select all features
    #fs = SelectKBest(score_func=f_classif, k=i)
    for label_num in range(len(y_train[0])):
        gc.collect()
        fs = SelectKBest(score_func=f_classif, k='all')
        # learn relationship from training data
        fs.fit(X_train, y_train[:,label_num])
        # transform train input data
        X_train_fs = fs.transform(X_train)
        # transform test input data
        X_test_fs = fs.transform(X_test)
        weights_for_label = fs.scores_
        #print(fs.scores_, fs.scores_.shape)
        selected_features_indxs = heapq.nlargest(i, range(len(weights_for_label)), weights_for_label.take)
        #print(set(selected_features_indxs))
        selected_features_indexes = selected_features_indexes.union(set(selected_features_indxs))
    return X_train_fs, X_test_fs , selected_features_indexes


fs_X_train,fs_X_test,selected_features_indexes = select_features(X_train, y_train, X_test, 20 ,selected_features_indexes)

print(len(selected_features_indexes),selected_features_indexes)

In [ ]:
from sklearn.feature_selection import chi2, SelectKBest, f_classif
import numpy as np

threshold = 200.5


selected_features = [] 
for label_num in range(y_train.shape[1]):
    selector = SelectKBest(f_classif, k='all')
    selector.fit(X_train, y_train[:,label_num])
    selected_features.append(list(selector.scores_))

#/ / MeanCS 
#selected_features = np.mean(selected_features, axis=0) > threshold
#// MaxCS
#selected_features = np.max(selected_features, axis=0) > threshold

# Этот вариант работает с multi-label. Но тут есть тонкость.
# Нужно отобрать features так, чтоб для каждого класса самые информативные присутствовали. 
# А мне кажется, что присутствуют те, в которых большая информативность хотя бы для одного класса.
# И возможна ситуация, что найдется класс, для которого не отобрались хоть сколько-то информативные фичи.

#len(selected_features[0])
# selected_features
# len(X_train[0][selected_features])

selected_features = heapq.nlargest(176, range(len(selected_features)), selected_features.__getitem__)
selected_features # i have no idea what happends here

In [ ]:
# VarianceThreshold is a simple baseline approach to feature selection. 
# It removes all features whose variance doesn’t meet some threshold

# Can we use it?
from sklearn.feature_selection import VarianceThreshold

import matplotlib.pyplot as plt

tr = np.arange(0.0, 1.0, 0.01)
s=[]
for ttt in tr:
    selector = VarianceThreshold(threshold=ttt)
    selector.fit_transform(X_train).shape
    
    s.append(selector.transform(X_test[0:1]).shape[1])

fig, ax = plt.subplots()
ax.plot(tr, s)

ax.set(xlabel='Threshold', ylabel='Feature remains',
       title='About as simple as it gets, folks')
ax.grid()

#fig.savefig("test.png")
plt.show()




In [ ]:
# Feature extraction using RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

feature_names = [f"feature {i}" for i in range(X_train[0].shape[0])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

import time
import numpy as np

start_time = time.time()
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

import pandas as pd

forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
# permutation_importance 
# Permutation feature importance is a model inspection technique that
#  can be used for any fitted estimator when the data is tabular.
# This is especially useful for non-linear or opaque estimators. 
# The permutation feature importance is defined to be the decrease
#  in a model score when a single feature value is randomly shuffled 1. 
# This procedure breaks the relationship between the feature and the target,
#  thus the drop in the model score is indicative of how much the model depends
#  on the feature. This technique benefits from being model agnostic and
#  can be calculated many times with different permutations of the feature.

from sklearn.inspection import permutation_importance
wanna_permutation_picture = False
if wanna_permutation_picture:
    start_time = time.time()
    result = permutation_importance(
        forest, X_test, y_test, n_repeats=2, random_state=42, n_jobs=4
    )
    elapsed_time = time.time() - start_time
    print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

    forest_importances = pd.Series(result.importances_mean, index=feature_names)




In [ ]:
if wanna_permutation_picture:
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
    ax.set_title("Feature importances using permutation on full model")
    ax.set_ylabel("Mean accuracy decrease")
    fig.tight_layout()
    plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import hamming_loss



clf = DecisionTreeClassifier(random_state=0)
len(X_train), X_train[0].shape

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import hamming_loss
import datetime
#rfc = RandomForestClassifier(n_jobs=7, n_estimators= 100)
#rfc.fit(X_train,y_train)
# (rfc.predict(X_train) - y_train).mean()
# (rfc.predict(X_test) - y_test).mean()

# Here we will test importance of  n_estimators&max_depth 
# Note, that X_train is only 0.1 of total TRAIN data set. So real fit will takes 10 times longer
for n_trees in [100 * i*i +1 for i in range(35)]:
    for depth in [i * i*i*i for i in range(4)]:
        time_start = datetime.datetime.now()
        if depth < 1:
            rfc = RandomForestClassifier(n_jobs=7, n_estimators= n_trees)
        else:
            rfc = RandomForestClassifier(n_jobs=7,max_depth = depth, n_estimators= n_trees)
        rfc.fit(X_train,y_train)
        pred = rfc.predict(X_test)

        print(f"For {n_trees} trees with depth {depth} RFC hamming loss = {hamming_loss(pred,y_test)} it takes {datetime.datetime.now() -time_start}")


In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rfc, open(filename, 'wb'))
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [ ]:
clf_pred = clf.predict(X_test)
rfc_pred = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import hamming_loss

hamming_loss(clf_pred,y_test), hamming_loss(rfc_pred,y_test)

In [ ]:
rfc2 = RandomForestClassifier(n_jobs=7, n_estimators= 100)
rfc2.fit(X_train,y_train)
rfc2_pred = rfc2.predict(X_test)
hamming_loss(rfc2_pred,y_test)

In [ ]:
rfc2.fit(X_train,y_train)
rfc2_pred = rfc2.predict(X_test)
hamming_loss(rfc2_pred,y_test)

In [ ]:
di = None # Find index of predictions with different results
for i in range(len(rfc2_pred)):
    if (rfc2_pred[i] != clf_pred[i]).any():
        di = i
        break

double_lable = None # Trying to find index of prediction with double lable
for i in range(len(rfc2_pred)):
    if (sum(rfc2_pred[i])>1):
        double_lable = i
        break

rfc_l = hamming_loss(rfc_pred,y_test)
clf_l = hamming_loss(clf_pred,y_test)
rfc2_l = hamming_loss(rfc2_pred,y_test)

rfc2_pred[di],clf_pred[di],di,len(rfc2_pred),rfc2_pred.shape,[rfc2_l,clf_l,rfc_l],'double label example = ', rfc2_pred[double_lable]



In [ ]:
# Let's create function for vote for predictions
def prediction_voting(predicts, voters_loss = None, voters_quality = None):
    if voters_loss:
        voters_quality = [ 1 / ll for ll in voters_loss]
    if not voters_quality:
        print("Need voters_loss or voters_quality")
        return
    pred_count = len(predicts)
    samples_count = len(predicts[0])
    labels_count = len(predicts[0][0])
    best_num = voters_quality.index(max(voters_quality)) # number of best predictor
    final_predicts = []
    for sample_num in range(samples_count ):
        s_preds = [predict[sample_num] for predict in predicts]
        pro = np.zeros(shape=(labels_count))
        cons = np.zeros(shape=(labels_count))
        res = np.zeros(shape=(labels_count))
        print("predicts ",s_preds)
        for pred_num in range(len(s_preds)):
            pred = s_preds[pred_num]
            qa = voters_quality[pred_num]
            pro = (pred == 1)*qa
            cons = -1 * (pred == 0)*qa
            
            res +=  pro + cons
            res2 = np.where(res < 0, 0, 1)
            #print("pros",pro,"\ncons ",cons, "\nsumm ", res,"\nresults ", res2)
        print("\nsumm ", res,"\nresults ", res2)
        
        if sum(res2) < 0.5:
            res2 = s_preds[best_num]
        final_predicts.append(res2)
    return final_predicts

    

vote_pred = prediction_voting([rfc2_pred,clf_pred,rfc_pred],[rfc2_l,clf_l,rfc_l])    

In [ ]:
# Comparing predictions before vote and after
[rfc2_l,clf_l,rfc_l], hamming_loss(vote_pred,y_test)

In [ ]:
# Check RandomForestClassifier criterion. Is it worth it to choose and change default.
for cla in ["gini","entropy","log_loss"]:
    test_rfc = RandomForestClassifier(n_jobs=7, n_estimators= 1000,criterion=cla)
    test_rfc.fit(X_train,y_train)
    test_rfc_pred = rfc.predict(X_test)
    print(cla," ",hamming_loss(test_rfc_pred,y_test))
# Looks like very close results.

In [ ]:
def labels2string(labels): # little magic to convert array like [0,1,1,0,0,1] to string like '1 2 5'
    return " ".join([ str(aa[0]) for aa in  np.argwhere(labels>0) ])


ppp = vote_pred[double_lable]
labels2string(ppp) # And simple test

In [ ]:
# Here i just want to test models and, may be, Tree_classifiers
#from sklearn.ensemble import RandomForest, ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from torchvision import models
import gc # garbage collector, can help you free memory


model_names = ["resnet50", 
"densenet161", 
"wide_resnet50_2",
"vgg13"]

trees_ensembles = { "RandomForest":RandomForestClassifier(n_jobs=7, n_estimators= 10000),
"ExtraTrees":ExtraTreesClassifier(n_jobs=7, n_estimators= 10000)}
mult_lab = train_metadata_df.iloc[:, 64:]

magic_test_results=dict()
for model_name in model_names:
    gc.collect()
    with open(model_name+'.fea', 'rb') as f:
        model_fea = pickle.load(f)

    train_metadata_df = pd.read_csv('dicom_metadata_train_merged.csv') # Took it from kaggle notebooks
    import torch
    model_magic_result = dict()
    model_fea = [ elem.numpy()[0] for elem in model_fea ]
    X_m_train, X_m_test, y_m_train, y_m_test = train_test_split(model_fea, mult_lab, test_size=0.33, random_state=42)
    gc.collect()
    for tree_name in trees_ensembles.keys():
        gc.collect()
        tree = trees_ensembles[tree_name]
        tree.fit(X_m_train,y_m_train)
        tree_pred = tree.predict(X_m_test)
        model_magic_result[tree_name] = hamming_loss(tree_pred,y_m_test)

    magic_test_results[model_name] = model_magic_result
    del model_fea
    del tree






In [ ]:
two_epoch = magic_test_results

In [ ]:
one_epoch